#**X-Ray Production**

**Observations:**

This experiment was peformed at the Institute of Physics, University of São Paulo in 2022 and it is part of modern physics laboratory classes. It was used NaCl to diffract X-Ray. 

**Experiment main goals:**

1.   To determine the X-Ray emission spectra of Mo tube (Count vs Energy): characteristic radiation and bremsstrahlung radiation.
2.   To determine the Planck constant from the bremsstrahlung radiation.
3.   To evaluate the relation between θmin and Emax.

**Datasets observations:**



*   The first dataset consists of 3 subsets at the angle range from θmin = 2.5̊  and θmax = 30̊ , Δt = 1s, Δθ = 0.1̊ , I = 1 mA and U = 35 kV.
*   The second dataset consists of 10 subsets with 5 different tension values, and for each tension value there is 2 values of Δt: 1s and 5s. All subsets were measured at the angle range from θmin = 2.5̊  and θmax = 12̊, I = 1 mA, and Δθ = 0.1̊. The tension values are: 18 kV, 25 kV, 28 kV, 30 kV and 35 kV.


**Experiment observations:**



*   The uncertainty in maximum energy (θmin) is 1%.
*   The equipament uncertainty in θ is 0.05̊.
*   The equipment software delivers the count  column divided by Δt, so in the case of sets where Δt = 5s we have to multiply the count column by 5 and then add the influence of time in the uncertainty calculation: σ(N) = $\sqrt{N}$/Δt.




**Importing libraries:**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.stats import linregress as lr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Reading datasets:**

In [ ]:
def read_c1(num):
  num = str(num)
  d1 = np.loadtxt(f'/content/drive/MyDrive/Raio X/Raio X/Dados/c1-{num}.txt', skiprows=1, dtype=np.str)
  d2 = np.char.replace(d1, ',', '.').astype(np.float64)
  return pd.DataFrame(d2, columns=['\u03B8 (°)','Count'])

def read_c2(tension, time):
  tensao = str(tension)
  tempo = str(time)
  d1 = np.loadtxt(f'/content/drive/MyDrive/Raio X/Raio X/Dados/c2-{tension}-{time}.txt', skiprows=1, dtype=np.str)
  d2 = np.char.replace(d1, ',', '.').astype(np.float64)
  d3 = pd.DataFrame(d2, columns=['\u03B8 (°)','Count'])
  if tempo == '5':
    d3['Count'] = d3['Count']*5
  return d3
 

#Dataset 1:
c1_1 = read_c1(1)  #subset 1
c1_2 = read_c1(2)  #subset 2
c1_3 = read_c1(3)  #subset 3

#Dataset 2:
c2_18_1 = read_c2(18,1) #U = 18 kV and Δt = 1s
c2_18_5 = read_c2(18,5) #U = 18 kV and Δt = 5s

c2_25_1 = read_c2(25,1) #U = 25 kV and Δt = 1s
c2_25_5 = read_c2(25,5) #U = 25 kV and Δt = 5s

c2_28_1 = read_c2(28,1) #U = 28 kV and Δt = 1s
c2_28_5 = read_c2(28,5) #U = 28 kV and Δt = 5s

c2_30_1 = read_c2(30,1) #U = 30 kV and Δt = 1s
c2_30_5 = read_c2(30,5) #U = 30 kV and Δt = 5s

c2_35_1 = read_c2(35,1) #U = 35 kV and Δt = 1s
c2_35_5 = read_c2(35,5) #U = 35 kV and Δt = 5s

**Checking datasets:**

In [ ]:
c1_1

#**Dataset 1:**

*   Comment qualitatively the observed spectra (continuous spectra and characteristic lines).
*   How to evaluate the experimental uncertainties?
*   Does the characteristic energies match to the expected ones?
*   Does the angles to n=2 and n=3 match to the expected ones?

**Analysis infos:**

*   The X-Ray detection obbey the Poisson distribution function. That is why the uncertanty is given by σ(N) = $\sqrt{N}$/Δt (N is the sum of the photons in time interval Δt).
*   To calculate characteristic peaks (kα and kβ) energies, first we obtain λ from θ values applied in Bragg equation **nλ = 2dsin(θ)**, where d = 0.282 nm (d value for NaCl), n=1. To calculate energy we calculate **E = hc/λ**.
   * **h = $4.14×10^{−15}$  eV s**
   * **c =  $3×10^8$  m/s**
*   To verify the angles for n = 2 and n = 3 we apply the λ found for n = 1 and compare them with the values from the graph curves. 
*   The expected values for kα and kβ are:
    * **kα = 17.44 eV**
    * **kβ = 19.60 eV**





**Merging the 3 subsets from dataset 1 to a dataset called c1:**

In [14]:
merge1c1 = pd.merge(c1_1, c1_2, how='outer', on = '\u03B8 (°)')
c1 = pd.merge(merge1c1, c1_3, how='outer', on = '\u03B8 (°)')
c1.columns = ['\u03B8 (°)', 'Count 1', 'Count 2', 'Count 3']

#Add the extra column summing the column values up and the column with uncertainties:
c1['Count Total'] = c1['Count 1'] + c1['Count 2'] + c1['Count 3']
c1['σ'] = np.sqrt(c1['Count Total'])

**Plotting the 3 subsets together:**

In [ ]:
p1 = go.Scatter(x=c1['\u03B8 (°)'], y=c1['Count 1'], mode='lines', name='Count 1')
p2 = go.Scatter(x=c1['\u03B8 (°)'], y=c1['Count 2'], mode='lines', name='Count 2')
p3 = go.Scatter(x=c1['\u03B8 (°)'], y=c1['Count 3'], mode='lines', name='Count 3')

plots1 = [p1,p2,p3]
layout1 = go.Layout(title='X-Ray detection - Dataset 1')
figure1 = go.Figure(data=plots1, layout=layout1)
figure1

**Plotting the total count:**

This is done to improve statistics.

In [ ]:
figure2 = px.line(c1, x='\u03B8 (°)', y='Count Total', title = 'X-Ray Detection - Total Count', error_y=c1['σ'])
figure2

**Selecting peaks to fit gaussian for n=1:**



In [ ]:
c1x = c1['θ (°)']
c1y = c1['Count Total']
c1_s = c1['σ']

#kβ:
c1x_b = c1x[(c1x >= 6.1) & (c1x <= 6.6)] # x values for kβ peak
c1y_b = c1y[(c1x >= 6.1) & (c1x <= 6.6)] # y values for kβ peak
c1_s_b = c1_s[(c1x >= 6.1) & (c1x <= 6.6)] # sigma count values for kβ peak
c1_b = pd.DataFrame({'x': c1x_b,'y': c1y_b, 'sy':c1_s_b})

#kα:
c1x_a = c1x[(c1x >= 6.9) & (c1x <= 7.6)] # x values for kα peak
c1y_a = c1y[(c1x >= 6.9) & (c1x <= 7.6)] # y values for kα peak
c1_s_a = c1_s[(c1x >= 6.9) & (c1x <= 7.6)] # sigma count values for kα peak 
c1_a = pd.DataFrame({'x': c1x_a,'y': c1y_a, 'sy':c1_s_a})

#Checking the .describe() function:

c1x_a.describe()

**The fitting was performed in two different ways: lmfit and scipy curve_fit**

#**Fitting with lmfit**

*  It is not included here the count uncertainties.

In [ ]:
!pip install lmfit

In [ ]:
from lmfit import Model

def gaussian(x, amp, cen, wid):
    return amp*np.exp(-(x-cen)**2/(2*wid**2))

#Fitting kβ:
b_gmodel = Model(gaussian)
b_result = b_gmodel.fit(c1_b['y'], x=c1x_b, amp=2700, cen=6.3, wid=0.2)
print(b_result.fit_report())


plt.plot(c1x_b, c1y_b, 'o')
plt.plot(c1x_b, b_result.init_fit, '--', label='Initial fit')
plt.plot(c1x_b, b_result.best_fit, '-', label='Best fit')
plt.title('Fitting kβ')
plt.legend()
plt.show()

In [ ]:
#Fitting kα:
a_gmodel = Model(gaussian)
a_result = a_gmodel.fit(c1y_a, x=c1x_a, amp=5700, cen=7.2, wid=0.1)

print(a_result.fit_report())

plt.plot(c1x_a, c1y_a, 'o')
plt.plot(c1x_a, a_result.init_fit, '--', label='initial fit')
plt.plot(c1x_a, a_result.best_fit, '-', label='best fit')
plt.title('Fitting kα')
plt.legend()
plt.show()

**Residuals:**

In [ ]:
# Residuals for kβ:
residuals_b1 = c1y_b - b_result.best_fit
plt.scatter(c1y_b, residuals_b1)
plt.axhline(y=0, c='red', ls='--')
plt.title('Residuals kβ, n=1')

In [ ]:
# Residuals for kα:
residuals_a1 = c1y_a - a_result.best_fit
plt.scatter(c1y_a, residuals_a1)
plt.axhline(y=0, c='red', ls='--')
plt.title('Residuals kα, n=1')

#**Fitting with curve_fit:**


*   It was included the count uncertainties.



In [ ]:
def gauss (x, amp, cen, wid):  #amp=ymax, cen=center of the curve, wid=width of the curve
    return amp*np.exp(-(x-cen)**2/(2*wid**2))

#Fitting kβ:
popt_b, pcov_b = curve_fit(gauss, c1_b['x'], c1_b['y'], sigma=c1_b['sy'], p0=[2000,6.3,0.2])
a = popt_b[0]
x0 = popt_b[1]
sigma = popt_b[2]
plt.plot(c1_b['x'],c1_b['y'], 'bo', label='Data')
plt.errorbar(c1_b['x'],c1_b['y'],yerr=c1_b['sy'], color='b')
plt.plot(c1_b['x'], gauss(c1_b['x'],*popt_b), color='r', label='Fit:' + '$\mathcal{N}$' + f'$(\mu \\approx {round(x0, 1)}, \sigma \\approx {round(sigma,2)})$')
plt.legend()
print('Params:','\n',popt_b, '\n','\n','Cov matrix:','\n', pcov_b)

In [ ]:
#Fitting kα:
popt_a, pcov_a = curve_fit(gauss, c1_a['x'], c1_a['y'], sigma=c1_a['sy'], p0=[5000,7.2,0.1])
a = popt_a[0]
x0 = popt_a[1]
sigma = popt_a[2]
plt.plot(c1_a['x'],c1_a['y'], 'go', label='Data')
plt.errorbar(c1_a['x'],c1_a['y'],yerr=c1_a['sy'], color='g')
plt.plot(c1_a['x'], gauss(c1_a['x'],*popt_a), color='purple', label='Fit:' + '$\mathcal{N}$' + f'$(\mu \\approx {round(x0, 1)}, \sigma \\approx {round(sigma,2)})$')
plt.legend()
print('Params:','\n',popt_a, '\n','\n','Cov matrix:','\n', pcov_a)

#**Comparing fits:**

In [ ]:
print('Result for kβ usando lmfit:','\n')
display(b_result)
print('\n')

print('Result for kα usando lmfit:','\n')
display(a_result)
print('\n')

print('Result for kβ usando curve_fit:','\n')
print('amp:',popt_b[0],'|','inc:', np.sqrt(pcov_b[0][0]),'|','relative error:',(np.sqrt(pcov_b[0][0])/popt_b[0])*100,'%','\n',
      'cen:',popt_b[1],'|','inc:', np.sqrt(pcov_b[1][1]),'|','relative error:',(np.sqrt(pcov_b[1][1])/popt_b[1])*100,'%','\n',
      'wid:',popt_b[2],'|','inc:', np.sqrt(pcov_b[2][2]),'|','relative error:',(np.sqrt(pcov_b[2][2])/popt_b[2])*100,'%')
print('\n')

print('Result for kα usando curve_fit:','\n')
print('amp:',popt_a[0],"|",'inc:', np.sqrt(pcov_a[0][0]),'|','relative error:',(np.sqrt(pcov_a[0][0])/popt_a[0])*100,'%','\n',
      'cen:',popt_a[1],'|','inc:', np.sqrt(pcov_a[1][1]),'|','relative error:',(np.sqrt(pcov_a[1][1])/popt_a[1])*100,'%','\n',
      'wid:',popt_a[2],'|','inc:', np.sqrt(pcov_a[2][2]),'|','relative error:',(np.sqrt(pcov_a[2][2])/popt_a[2])*100,'%')
print('\n')


**$R^2$:**

In [ ]:
def rsquared(y, yfit):
    slope, intercept, r_value, p_value, std_err = lr(y, yfit)
    return r_value**2

# R2 values for lmfit:
b_r2_lm = rsquared(c1_b['y'], b_result.best_fit)
a_r2_lm = rsquared(c1_a['y'], a_result.best_fit)

# R2 values for curve_fit():
b_r2_cf = rsquared(c1_b['y'], gauss(c1_b['x'],*popt_b))
a_r2_cf = rsquared(c1_a['y'], gauss(c1_a['x'],*popt_a))

print(a_r2_cf, a_r2_lm, b_r2_cf,b_r2_lm)

#**Applying the parameters fitted with lmft to calculate λ:**


*   OBS.: Comparing the fitting methods it is possible to conclude there is no big difference between parameters and its uncertainties values looking to relative errors and $R^2$.



In [29]:
#Calculating λ from fitted θ values. These values are the 'cen' parameter,
#which is the mean of gaussian curve fitted. Let's apply Bragg equation: nλ=2dsen(θ) => λ = 2dsen(θ)

d = 0.282*10**(-9) #meters
#kβ:
lambda_b = 2*d*np.sin(6.3511*(np.pi/180))

#Para kα:
lambda_a = 2*d*np.sin(7.18760*(np.pi/180))

**Calculating energy from λ and comparing them to the expected values:**

Energy equation: E = hc/λ

In [30]:
h = 4.14*10**(-15) #eV s
c = 3*10**8 #m/s

#Expected values:
E_ka_t = 17440 #eV  
E_kb_t = 19600 #eV

#kβ:
E_b = (h*c)/lambda_b
desvio_a = (abs(E_b - E_kb_t)/E_kb_t)*100 #desvio percentual de 0.92%

#kα:
E_a = (h*c)/lambda_a
desvio_b = (abs(E_a - E_ka_t)/E_ka_t)*100 #desvio percentual de 1.57%

**Verifying the angles for n = 2 and n = 3:**

n = 2:

In [ ]:
#kα:
theta_a_n2 = np.arcsin(lambda_a/d)*(180/np.pi) #degrees

#kβ:
theta_b_n2 = np.arcsin(lambda_b/d)*(180/np.pi) #degrees

#I did quick check using the plotted figure.

n = 3:

In [ ]:
#kα:
theta_a_n3 = np.arcsin((3*lambda_a)/(2*d))*(180/np.pi) #degrees

#kβ:
theta_b_n3 = np.arcsin((3*lambda_b)/(2*d))*(180/np.pi) #degrees

#I did quick check using the plotted figure.

#**Dataset 2:**

##**To be continued**